# Laboratorio 2025
### Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1


In [ ]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
from pydistsim.restrictions.topological import Connectivity
from pydistsim.restrictions.knowledge import InitialDistinctValues

from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger
from pydistsim.network.behavior import NetworkBehaviorModel
from pydistsim.gui import drawing as draw
%matplotlib inline
from matplotlib import pyplot as plt


## Parte 1: 

### 1. Implementación básica del problema
- Simule un conjunto de n generales (nodos) que deben decidir si atacar o retirarse.
- Permita que hasta f generales sean bizantinos, es decir, que puedan enviar mensajes contradictorios.
- Los generales leales deberán acordar una decisión común, cumpliendo las condiciones de consistencia y validez.


### 2. Protocolos de comunicación
- Implemente el protocolo recursivo propuesto por Lamport, Shostak y Pease (OM(f)).
- Muestre como crece la complejidad en mensajes según la cantidad de fallos f. Evalúe el caso f = 1 y luego f = 2.
- Mida el número total de mensajes intercambiados y el tiempo necesario para llegar al consenso.

### 3. Análisis
- Determine experimentalmente el número mínimo de nodos necesario para alcanzar consenso frente a f fallos bizantinos.
- Verifique la condición teórica n ≥ 3f + 1.

## Otros

### Implementacion

In [ ]:
# By the start of AllBridges, every x should already know: 
#     Its children in Tr, 
#     Its parent in Tr, 
#     Its associated labels α(x) = (a, b) in Tr, 
#     Its neighbors in G.



class Algorithm(NodeAlgorithm):
    default_params = {
        "Notify" : "Notify",
    }

    class Status(StatusValues):
        INITIATOR = "INITIATOR"

    S_init = (Status.INITIATOR, Status.IDLE)
    S_term = ()

    algorithm_restrictions = (
        BidirectionalLinks,
        Connectivity, 
        TotalReliability, 
        InitialDistinctValues
    )

    def costo(self, node: NodeAccess, neigh_id):
        # No encontre la forma de asociar costos
        return 1

    def error(self, method : str, message: Message):
        msj = 'Unexpected message in ' + method + message.header + " from " + str(message.source) + " , content: " + str(message.data)
        raise Exception(msj)
    
    def initializer(self):
        self.apply_restrictions()
        for node in self.network.nodes():
            node.status = self.Status.IDLE
            node.memory["source"] = False
        ini_node = self.network.nodes_sorted()[0]
        ini_node.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=ini_node))
        ini_node.status = self.Status.INITIATOR

 
    @Status.INITIATOR
    def spontaneously(self, node: NodeAccess, message: Message):
        node.memory["source"] = True
        self.send(
            node, 
            data=None,
            destination=list(node.neighbors()),
            header=self.default_params["Notify"]
        )

    @Status.INITIATOR
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["ack"]:
            pass
        else:
            self.error("INITIATOR::receiving", message)


    @Status.DONE
    def default(self, *args, **kwargs):    
        pass

### Ejecución


In [ ]:
net_gen = NetworkGenerator(3, directed=False)
net = net_gen.generate_random_network()
sim = Simulation(net, check_restrictions=True)
sim.algorithms = (Algorithm,)
set_log_level(LogLevels.INFO)
enable_logger()

fig = draw.draw_current_state(sim)
fig

In [ ]:
sim.run(1)

fig = draw.draw_current_state(sim)
fig


### Otros

In [ ]:
sim.reset()
plt.close()